# Step 1: Data preprocessing

We will read the CSV files generate by `preprocess_dataset.py`, format the data as appropriate and split the training set into training and validation sets.

In [1]:
import pandas as pd
import numpy as np

# Load training and test data
train_data = pd.read_csv('data/train_data.csv')
test_data = pd.read_csv('data/test_data.csv')

# Define input features (X) and output target (y)
X_train = train_data.iloc[:, :-1].values  # All columns except the last (inputs)
y_train = train_data.iloc[:, -1].values  # Last column (output)
y_train = y_train.reshape(-1, 1).flatten()

X_test = test_data.iloc[:, :-1].values  # All columns except the last (inputs)
y_test = test_data.iloc[:, -1].values  # Last column (output)
y_test = y_test.reshape(-1, 1).flatten()

# Step 2: Implement and train BP-based Neural Network

Now we will use the `NeuralNet` implementation with varying hyperparameters, to see which combination performs better.

In [7]:
from NeuralNet import NeuralNet

# Define hyperparameter combinations
param_grid = [
    {'layers': [16, 9, 1], 'epochs': 50, 'lr': 0.01, 'momentum': 0.9, 'activation': 'sigmoid'},
    {'layers': [16, 12, 6, 1], 'epochs': 80, 'lr': 0.02, 'momentum': 0.8, 'activation': 'relu'},
    {'layers': [16, 9, 5, 1], 'epochs': 100, 'lr': 0.01, 'momentum': 0.5, 'activation': 'tanh'},
    # TODO: Add more hyperparameter combinations
]

results = []

for params in param_grid:
    model = NeuralNet(
        layers=params['layers'],
        epochs=params['epochs'],
        lr=params['lr'],
        momentum=params['momentum'],
        activation=params['activation'],
        validation_split=0.2
    )

    # Fit the model
    model.fit(X_train, y_train)

    # Evaluate on test set
    y_test_pred = model.predict(X_test)

    mse = np.mean((y_test - y_test_pred) ** 2)  # Mean Squared Error
    mae = np.mean(np.abs(y_test - y_test_pred))  # Mean Absolute Error
    mape = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100  # Mean Absolute Percentage Error

    # Record results
    results.append({
        'Number of layers': len(params['layers']),
        'Layer structure': params['layers'],
        'Num epochs': params['epochs'],
        'Learning rate': params['lr'],
        'Momentum': params['momentum'],
        'Activation function': params['activation'],
        'MSE': mse,
        'MAE': mae,
        'MAPE': mape,
    })

results_df = pd.DataFrame(results)
print(results_df)

KeyboardInterrupt: 